In [17]:
import numpy as np
import pandas as pd 
import os
import sys
import cv2 

import os
import datetime
import copy
import re
import yaml
import uuid
import warnings
import time
import inspect
import random

In [18]:
DATA_PATH = '/home/modeling/data/'
VIDEO_PATH = os.path.join(DATA_PATH, 'video')
ANNOTATION_PATH = os.path.join(DATA_PATH, 'annotation')

In [19]:
columns = ['video_name']

In [20]:
train_df = pd.read_csv(os.path.join(ANNOTATION_PATH, 'Train_Annotation.txt'), header=None, names=columns)
test_df = pd.read_csv(os.path.join(ANNOTATION_PATH, 'Test_Annotation.txt'), header=None, names=columns)

train_df

,video_name
0,Vandalism/Vandalism048_x264.mp4 7183
1,Training_Normal_Videos_Anomaly/Normal_Videos82...
2,Robbery/Robbery003_x264.mp4 984
3,Assault/Assault046_x264.mp4 952
4,Training_Normal_Videos_Anomaly/Normal_Videos57...
...,...
1605,Burglary/Burglary073_x264.mp4 2769
1606,Training_Normal_Videos_Anomaly/Normal_Videos05...
1607,Training_Normal_Videos_Anomaly/Normal_Videos24...
1608,Burglary/Burglary022_x264.mp4 2838


In [21]:
len(train_df), len(test_df)

(1610, 290)

In [22]:
anomal_list = ['Arson', 'Assault', 'Stealing', 'Training_Normal_Videos_Anomaly']

new_tr_video = []
new_te_video = []

for data in train_df['video_name']:
    ab_type, video_name = data.split('/')
    if ab_type in anomal_list: 
        new_tr_video.append(ab_type + '/' + video_name)
        
for data in test_df['video_name']:
    ab_type, video_name = data.split('/')
    if ab_type in anomal_list: 
        new_te_video.append(ab_type + '/' + video_name)
        
len(new_tr_video), len(new_te_video)

(983, 17)

In [23]:
new_train_df = pd.DataFrame(new_tr_video, columns=['video_name'])
new_test_df = pd.DataFrame(new_te_video, columns=['video_name'])

In [24]:
new_test_df

,video_name
0,Arson/Arson022_x264.mp4 8640 Arson 3500 4000 -...
1,Stealing/Stealing036_x264.mp4 2503 Stealing 12...
2,Arson/Arson018_x264.mp4 842 Arson 270 600 -1 -1
3,Arson/Arson011_x264.mp4 1266 Arson 150 420 680...
4,Assault/Assault011_x264.mp4 2288 Assault 375 9...
5,Arson/Arson035_x264.mp4 1437 Arson 600 900 -1 -1
6,Assault/Assault006_x264.mp4 8096 Assault 1185 ...
7,Assault/Assault010_x264.mp4 16177 Assault 1133...
8,Stealing/Stealing058_x264.mp4 4991 Stealing 57...
9,Arson/Arson010_x264.mp4 3159 Arson 885 1230 -1...


In [25]:
new_train_df.to_csv(os.path.join(ANNOTATION_PATH, 'Train_annotation.csv'), header=0, index=None)
new_test_df.to_csv(os.path.join(ANNOTATION_PATH, 'Test_annotaion.csv'), header=0, index=None)

#### DataFrame 재구성

In [2]:
columns = ['video_name']

In [ ]:
train_df = pd.read_csv(os.path.join(ANNOTATION_PATH, 'Anomaly_Train.txt'), header=None,names=columns) 
test_df = pd.read_csv(os.path.join(ANNOTATION_PATH, 'Anomaly_Test.txt'), header=None, names=columns)

In [ ]:
anomal_list = ['Arson', 'Assault', 'Stealing']

new_tr_video = []
new_te_video = []

for data in train_df['video_name']:
    ab_type, video_name = data.split('/')
    if ab_type in anomal_list: 
        new_tr_video.append(ab_type + '/' + video_name)

for data in test_df['video_name']:
    ab_type, video_name = data.split('/')
    if ab_type in anomal_list: 
        new_te_video.append(ab_type + '/' + video_name)
        
len(new_tr_video), len(new_te_video)

In [ ]:
new_train_df = pd.DataFrame(new_tr_video, columns=['video_name'])
new_test_df = pd.DataFrame(new_te_video, columns=['video_name'])

len(new_train_df), len(new_test_df)

In [ ]:
new_train_df.head(3)

In [ ]:
new_train_df.to_csv(os.path.join(ANNOTATION_PATH, 'new_train_df.csv'), header=0)
new_test_df.to_csv(os.path.join(ANNOTATION_PATH, 'new_test_df.csv'), header=0)

### Normal_Video 파일 생성 
- 파일명: Normal_Train.txt, Normal_Test.txt
- 파일 내용: Normal/Normal_videos002_x264.mp4

In [30]:
file_list = os.listdir(os.path.join(VIDEO_PATH, 'Normal'))

In [31]:
columns = ['video_name']

In [32]:
len(file_list), type(file_list)

(287, list)

In [33]:
random.shuffle(file_list)

In [39]:
len(file_list)

287

#### Video 시간 구하기 (10분 이내의 Video들만 계산)  -> 18,000 frames 미만

In [35]:
def get_total_frames(video_name):   # video_name: Abuse/Abuse005_.mp4 
    video = os.path.join(VIDEO_PATH, video_name)

    cap = cv2.VideoCapture(video)
    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    
    return total_frames

In [36]:
train_df = []
test_df = []

In [37]:
int(len(file_list) * 0.8)

229

In [38]:
for idx, data in enumerate(file_list):
    if idx >= int(len(file_list) * 0.9):
        if get_total_frames('Normal/' + data) <= 18000: 
            test_df.append('Normal/' + data)
    else:
        if get_total_frames('Normal/' + data) <= 18000: 
            train_df.append('Normal/' + data)
        
len(train_df), len(test_df)

[mov,mp4,m4a,3gp,3g2,mj2 @ 0x2af3580] moov atom not found


(236, 26)

In [40]:
normal_train_df = pd.DataFrame(train_df, columns=columns)

In [42]:
normal_test_df = pd.DataFrame(test_df, columns=['video_name'])

In [43]:
normal_test_df.head(3)

,video_name
0,Normal/Normal_Videos336_x264.mp4
1,Normal/Normal_Videos315_x264.mp4
2,Normal/Normal_Videos359_x264.mp4


In [44]:
normal_train_df.to_csv(os.path.join(ANNOTATION_PATH,'Normal_Train.txt'), header=0, index=None)
normal_test_df.to_csv(os.path.join(ANNOTATION_PATH,'Normal_Test.txt'), header=0, index=None)

### 이상행동 csv 로드 

In [45]:
train_df = pd.read_csv(os.path.join(ANNOTATION_PATH, 'new_train_df.csv'), header=None,names=columns) 
test_df = pd.read_csv(os.path.join(ANNOTATION_PATH, 'new_test_df.csv'), header=None,names=columns) 

In [46]:
len(train_df), len(test_df)

(183, 17)

In [50]:
new_anomal_train = [] 
new_anomal_test = [] 

In [51]:
for data in train_df['video_name']:
    if get_total_frames(data) <= 18000: 
        new_anomal_train.append(data)

for data in test_df['video_name']:
    if get_total_frames(data) <= 18000: 
        new_anomal_test.append(data)

In [47]:
len(normal_train_df), len(normal_test_df)

(236, 26)

In [52]:
len(new_anomal_train), len(new_anomal_test)

(179, 17)

In [53]:
train_df = pd.DataFrame(new_anomal_train, columns=columns)
test_df = pd.DataFrame(new_anomal_test, columns=columns)

In [54]:
len(train_df), len(test_df)

(179, 17)

In [56]:
train_df = pd.concat([train_df, normal_train_df])
test_df = pd.concat([test_df, normal_test_df])

In [57]:
len(train_df), len(test_df)

(415, 43)

In [59]:
train_df.head(5)

,video_name
0,Arson/Arson001_x264.mp4
1,Arson/Arson002_x264.mp4
2,Arson/Arson003_x264.mp4
3,Arson/Arson005_x264.mp4
4,Arson/Arson006_x264.mp4


In [60]:
train_df.to_csv(os.path.join(ANNOTATION_PATH,'Anomaly_Train.csv'), header=0, index=None)
test_df.to_csv(os.path.join(ANNOTATION_PATH,'Anomaly_Test.csv'), header=0, index=None)